In [7]:
import os
import streamlit as st

from dotenv import load_dotenv

from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_ollama import ChatOllama, OllamaEmbeddings
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder
from langchain.prompts.chat import ChatPromptTemplate
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [3]:
loader = WebBaseLoader('https://aws.amazon.com/what-is/ai-agents/')
documents = loader.load()

In [5]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents = text_splitter.split_documents(documents)

In [8]:
embedding = OllamaEmbeddings(model='mxbai-embed-large:335m')

In [9]:
vectordb = Chroma.from_documents(documents=documents, embedding=embedding)

In [10]:
vectordb.similarity_search_with_score('what is ai agent')

[(Document(metadata={'description': 'What is AI agents how and why businesses use AI agents, and how to use AI agents with AWS.', 'language': 'en-US', 'source': 'https://aws.amazon.com/what-is/ai-agents/', 'title': 'What are AI Agents?- Agents in Artificial Intelligence Explained - AWS'}, page_content='What are AI Agents?\n\nAn artificial intelligence (AI) agent is a software program that can interact with its environment, collect data, and use the data to perform self-determined tasks to meet predetermined goals. Humans set goals, but an AI agent independently chooses the best actions it needs to perform to achieve those goals. For example, consider a contact center AI agent that wants to resolves customer queries. The agent will automatically ask the customer different questions, look up information in internal documents, and respond with a solution. Based on the customer responses, it determines if it can resolve the query itself or pass it on to a human.\nLearn more about what is a

In [11]:
retriever = vectordb.as_retriever()

In [13]:
llm = ChatOllama(model='llama3.2:3b')

In [15]:
system_prompt = (
    "Give a chat history and the latest user question"
    "which might reference context in the chat history,"
    "formulate a stadalone question which can be understood"
    "without the chat history. Do Not answer the question"
    'just reformulate it if needed and otherwise return it as is.'
)

In [31]:
template = """Answer the question based only on the following context:

{context}

Question: {input}
"""

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ('system', system_prompt),
        MessagesPlaceholder('chat_history'),
        ('human', template)
    ]
)

In [28]:
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ('system', system_prompt),
        MessagesPlaceholder('chat_history'),
        ('human', '{input}')
    ]
)

In [29]:
history_aware_retriever = create_history_aware_retriever(llm, retriever, contextualize_q_prompt)

In [30]:
history_aware_retriever.invoke({'input' : 'what is ai agents'})

[Document(metadata={'description': 'What is AI agents how and why businesses use AI agents, and how to use AI agents with AWS.', 'language': 'en-US', 'source': 'https://aws.amazon.com/what-is/ai-agents/', 'title': 'What are AI Agents?- Agents in Artificial Intelligence Explained - AWS'}, page_content='What are AI Agents?\n\nAn artificial intelligence (AI) agent is a software program that can interact with its environment, collect data, and use the data to perform self-determined tasks to meet predetermined goals. Humans set goals, but an AI agent independently chooses the best actions it needs to perform to achieve those goals. For example, consider a contact center AI agent that wants to resolves customer queries. The agent will automatically ask the customer different questions, look up information in internal documents, and respond with a solution. Based on the customer responses, it determines if it can resolve the query itself or pass it on to a human.\nLearn more about what is ar

In [32]:
document_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, document_chain)

In [33]:
store = {}

def get_session_memory(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()

    else:
        return store[session_id]

In [34]:
runnable = RunnableWithMessageHistory(
    rag_chain,
    get_session_memory,
    input_messages_key='input',
    history_messages_key='chat_history',
    output_messages_key='answer'
)

In [36]:
response = runnable.invoke(
    {'input' : "what is ai agents"},
    config = {'configurable' : {'session_id' : 'chat1'}}
)
response

{'input': 'what is ai agents',
 'chat_history': [],
 'context': [Document(metadata={'description': 'What is AI agents how and why businesses use AI agents, and how to use AI agents with AWS.', 'language': 'en-US', 'source': 'https://aws.amazon.com/what-is/ai-agents/', 'title': 'What are AI Agents?- Agents in Artificial Intelligence Explained - AWS'}, page_content='What are AI Agents?\n\nAn artificial intelligence (AI) agent is a software program that can interact with its environment, collect data, and use the data to perform self-determined tasks to meet predetermined goals. Humans set goals, but an AI agent independently chooses the best actions it needs to perform to achieve those goals. For example, consider a contact center AI agent that wants to resolves customer queries. The agent will automatically ask the customer different questions, look up information in internal documents, and respond with a solution. Based on the customer responses, it determines if it can resolve the que

In [37]:
response = runnable.invoke(
    {'input' : "what is its applications"},
    config = {'configurable' : {'session_id' : 'chat1'}}
)
response

{'input': 'what is its applications',
 'chat_history': [HumanMessage(content='what is ai agents', additional_kwargs={}, response_metadata={}),
  AIMessage(content='An AI agent is a software program that can interact with its environment, collect data, and use the data to perform self-determined tasks to meet predetermined goals.', additional_kwargs={}, response_metadata={})],
 'context': [Document(metadata={'description': 'What is AI agents how and why businesses use AI agents, and how to use AI agents with AWS.', 'language': 'en-US', 'source': 'https://aws.amazon.com/what-is/ai-agents/', 'title': 'What are AI Agents?- Agents in Artificial Intelligence Explained - AWS'}, page_content='What are AI Agents?\n\nAn artificial intelligence (AI) agent is a software program that can interact with its environment, collect data, and use the data to perform self-determined tasks to meet predetermined goals. Humans set goals, but an AI agent independently chooses the best actions it needs to perfo

In [38]:
store['chat1'].messages

[HumanMessage(content='what is ai agents', additional_kwargs={}, response_metadata={}),
 AIMessage(content='An AI agent is a software program that can interact with its environment, collect data, and use the data to perform self-determined tasks to meet predetermined goals.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='what is its applications', additional_kwargs={}, response_metadata={}),
 AIMessage(content="It's applications.\n\nExamples include:\n\n*   Self-driving cars\n*   Chatbots\n*   Personal assistants\n*   Robotic systems\n*   Contact center AI agents\n*   Autonomous drones\n*   Smart home devices\n*   Healthcare diagnosis and treatment recommendations\n*   Financial portfolio management\n*   Virtual customer service", additional_kwargs={}, response_metadata={})]